# To Be or Not To Be

In [388]:
import numpy as np

In [389]:
import pandas as pd

In [390]:
import matplotlib.pyplot as plt

In [391]:
dfShake=pd.read_csv("file://localhost/Users/anissakhan/Dropbox/Nissa_stuff/KU_CS/EECS731/731-Project-2-Khan/Data/RawData/Shakespeare_data.csv")

In [392]:
dfShake.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"


## Data Cleaning

If Player doesn't exist (Player=NaN), delete those observations, because the associated PlayerLine is not spoken by anyone (example: PlayerLine=ACT I).

If ActSceneLine doesn't exist (ActSceneLine=NaN), delete those observations, because the associated PlayerLine is a stage direction and therefore not spoken by anyone (example: ActSceneLine=SCENE III. London. The palace.).

In the dropna command, inplace=True because we want the dataframe itself to be changed (as opposed to a copy of the dataframe)

In [393]:
dfShake.dropna(subset = ["Player"], inplace=True)

In [394]:
dfShake.dropna(subset = ["ActSceneLine"], inplace=True)

Verify there are no more missing values in the dataset. Since the number of entries matches the non-null count for all variables, we know there are no more missing values.

In [395]:
dfShake.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105152 entries, 3 to 111394
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Dataline          105152 non-null  int64  
 1   Play              105152 non-null  object 
 2   PlayerLinenumber  105152 non-null  float64
 3   ActSceneLine      105152 non-null  object 
 4   Player            105152 non-null  object 
 5   PlayerLine        105152 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 5.6+ MB


In [396]:
dfShake.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"
5,6,Henry IV,1.0,1.1.3,KING HENRY IV,And breathe short-winded accents of new broils
6,7,Henry IV,1.0,1.1.4,KING HENRY IV,To be commenced in strands afar remote.
7,8,Henry IV,1.0,1.1.5,KING HENRY IV,No more the thirsty entrance of this soil


In [397]:
dfShake=dfShake.reset_index()

In [398]:
dfShake

,index,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
1,4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"
2,5,6,Henry IV,1.0,1.1.3,KING HENRY IV,And breathe short-winded accents of new broils
3,6,7,Henry IV,1.0,1.1.4,KING HENRY IV,To be commenced in strands afar remote.
4,7,8,Henry IV,1.0,1.1.5,KING HENRY IV,No more the thirsty entrance of this soil
...,...,...,...,...,...,...,...
105147,111390,111391,A Winters Tale,38.0,5.3.179,LEONTES,"Is troth-plight to your daughter. Good Paulina,"
105148,111391,111392,A Winters Tale,38.0,5.3.180,LEONTES,"Lead us from hence, where we may leisurely"
105149,111392,111393,A Winters Tale,38.0,5.3.181,LEONTES,Each one demand an answer to his part
105150,111393,111394,A Winters Tale,38.0,5.3.182,LEONTES,Perform'd in this wide gap of time since first


Convert Player and PlayerLine to string

In [399]:
dfShake.dtypes

index                 int64
Dataline              int64
Play                 object
PlayerLinenumber    float64
ActSceneLine         object
Player               object
PlayerLine           object
dtype: object

In [400]:
dfShake['Player'] = dfShake['Player'].astype("string")
dfShake['PlayerLine'] = dfShake['PlayerLine'].astype("string")

In [401]:
dfShake.dtypes

index                 int64
Dataline              int64
Play                 object
PlayerLinenumber    float64
ActSceneLine         object
Player               string
PlayerLine           string
dtype: object

There is a problem with the data: The player name "ANTIPHOLUS OF EPHESUS" has been inadvertently split. "ANTIPHOLUS" is split off from the rest of the name and placed incorrectly in PlayerLine

In [402]:
dfShake.PlayerLine = dfShake.PlayerLine.str.strip()

In [403]:
for i in range(0,(len(dfShake['PlayerLine']))):
    if(dfShake['PlayerLine'][i] == 'ANTIPHOLUS'):
        print(dfShake['PlayerLine'][i], dfShake['Dataline'][i])

ANTIPHOLUS 22600
ANTIPHOLUS 22612
ANTIPHOLUS 22624
ANTIPHOLUS 22629
ANTIPHOLUS 22651
ANTIPHOLUS 22657
ANTIPHOLUS 22668
ANTIPHOLUS 22673
ANTIPHOLUS 22679
ANTIPHOLUS 22689
ANTIPHOLUS 22694
ANTIPHOLUS 22700
ANTIPHOLUS 22837
ANTIPHOLUS 22852
ANTIPHOLUS 22856
ANTIPHOLUS 22862
ANTIPHOLUS 22868
ANTIPHOLUS 22883
ANTIPHOLUS 22886
ANTIPHOLUS 22890
ANTIPHOLUS 22897
ANTIPHOLUS 22900
ANTIPHOLUS 22904
ANTIPHOLUS 22907
ANTIPHOLUS 22910
ANTIPHOLUS 22914
ANTIPHOLUS 22918
ANTIPHOLUS 22922
ANTIPHOLUS 22926
ANTIPHOLUS 22930
ANTIPHOLUS 22935
ANTIPHOLUS 22938
ANTIPHOLUS 22942
ANTIPHOLUS 22945
ANTIPHOLUS 22948
ANTIPHOLUS 22951
ANTIPHOLUS 22956
ANTIPHOLUS 22961
ANTIPHOLUS 22966
ANTIPHOLUS 23007
ANTIPHOLUS 23016
ANTIPHOLUS 23022
ANTIPHOLUS 23026
ANTIPHOLUS 23030
ANTIPHOLUS 23046
ANTIPHOLUS 23062
ANTIPHOLUS 23065
ANTIPHOLUS 23081
ANTIPHOLUS 23094
ANTIPHOLUS 23109
ANTIPHOLUS 23115
ANTIPHOLUS 23120
ANTIPHOLUS 23124
ANTIPHOLUS 23127
ANTIPHOLUS 23143
ANTIPHOLUS 23147
ANTIPHOLUS 23151
ANTIPHOLUS 23176
ANTIPHOLUS 231

In [404]:
for i in range(0,(len(dfShake['PlayerLine']))):
    if(dfShake['PlayerLine'][i] == 'ANTIPHOLUS'):
        dfShake.drop([i],inplace=True, axis=0)

In [405]:
dfShake=dfShake.reset_index()

In [406]:
for i in range(0,len(dfShake['Player'])):
    if(dfShake['Player'][i] == 'OF EPHESUS'):
        dfShake['Player'][i]='ANTIPHOLUS OF EPHESUS'

There is also a problem with the player "DROMIO OF SYRACUSE". His whole name isn't stored in Player consistently

In [412]:
for i in range(0,(len(dfShake['Player']))):
    if(dfShake['Player'][i] == 'OF SYRACUSE'):
        print(dfShake['Player'][i], dfShake['Dataline'][i])

In [411]:
for i in range(0,len(dfShake['Player'])):
    if(dfShake['Player'][i] == 'OF SYRACUSE'):
        dfShake['Player'][i]='DROMIO OF SYRACUSE'

In [386]:
#dfShake.to_csv('test.csv')

Combine player lines so that the entire line is contained in a single observation

In [216]:
dfShake=dfShake.groupby(['Play','PlayerLinenumber', 'Player'], as_index = False).agg({'PlayerLine': ' '.join})

In [229]:
dfShake

,Play,PlayerLinenumber,Player,PlayerLine
0,A Comedy of Errors,1.0,ADRIANA,"Neither my husband nor the slave return'd, Tha..."
1,A Comedy of Errors,1.0,AEGEON,"Proceed, Solinus, to procure my fall And by th..."
2,A Comedy of Errors,1.0,ANGELO,"I am sorry, sir, that I have hinder'd you, But..."
3,A Comedy of Errors,1.0,First Merchant,"Therefore give out you are of Epidamnum, Lest ..."
4,A Comedy of Errors,1.0,LUCIANA,And may it be that you have quite forgot A hus...
...,...,...,...,...
22261,macbeth,61.0,MALCOLM,Dispute it like a man.
22262,macbeth,62.0,MACDUFF,"I shall do so, But I must also feel it as a ma..."
22263,macbeth,63.0,MALCOLM,Be this the whetstone of your sword: let grief...
22264,macbeth,64.0,MACDUFF,"O, I could play the woman with mine eyes And b..."


## Feature Engineering

Create new variable that contains the number of words in each player line

In [189]:
numWordsInLine = []
for x in range(0,len(dfShake['PlayerLine'])):
    numWordsInLine.append(len(dfShake['PlayerLine'][x].split()))
    #dfShake.assign(numWordsInLine=len(dfShake['PlayerLine'][x].split()))
dfShake['numWordsInLine'] = numWordsInLine

In [190]:
dfShake.head()

,Play,PlayerLinenumber,Player,PlayerLine,numWordsInLine
0,A Comedy of Errors,1.0,ADRIANA,"Neither my husband nor the slave return'd, Tha...",71
1,A Comedy of Errors,1.0,AEGEON,"Proceed, Solinus, to procure my fall And by th...",16
2,A Comedy of Errors,1.0,ANGELO,"I am sorry, sir, that I have hinder'd you, But...",25
3,A Comedy of Errors,1.0,First Merchant,"Therefore give out you are of Epidamnum, Lest ...",61
4,A Comedy of Errors,1.0,LUCIANA,And may it be that you have quite forgot A hus...,227


Summary statistics for the number of words per player line variable:

In [153]:
dfShake['numWordsInLine'].describe()

count    22266.000000
mean        35.426480
std         53.489294
min          1.000000
25%          8.000000
50%         17.000000
75%         41.000000
max       1555.000000
Name: numWordsInLine, dtype: float64

Check how many times there are 1 word lines:

In [191]:
count=0
for i in range(0,len(dfShake['PlayerLine'])):
    if(dfShake['numWordsInLine'][i] == 1):
        #print(dfShake.iloc[i])
        count=count+1

print('Count of 1 word lines =', count) 

Count of 1 word lines = 215


Check how many times there are 1555 word lines:

In [193]:
count=0
for i in range(0,len(dfShake['PlayerLine'])):
    if(dfShake['numWordsInLine'][i] == 1555):
        print(dfShake.iloc[i])
        count=count+1

print('Count of 1555 word lines =', count) 

Play                                                          Henry V
PlayerLinenumber                                                    1
Player                                                         Chorus
PlayerLine          O for a Muse of fire, that would ascend The br...
numWordsInLine                                                   1555
Name: 6221, dtype: object
Count of 1 word lines = 1


Check how many times there are < and >= an average # word lines (35):

In [194]:
count=0
for i in range(0,len(dfShake['PlayerLine'])):
    if(dfShake['numWordsInLine'][i] < 35):
        #print(dfShake.iloc[i])
        count=count+1

print('Count of <avg lines =', count) 

Count of <avg lines = 15750


In [195]:
count=0
for i in range(0,len(dfShake['PlayerLine'])):
    if(dfShake['numWordsInLine'][i] >= 35):
        #print(dfShake.iloc[i])
        count=count+1

print('Count of >=avg lines =', count) 

Count of >=avg lines = 6516
